In [1]:
!pip install sklearn -q
!pip install spacy -q

  Preparing metadata (setup.py) ... done


In [2]:
# Source : https://www.kaggle.com/code/achintyatripathi/spam-or-not-using-spacy-98-recall-score/notebook

In [3]:
!python -m spacy download en_core_web_sm

2023-04-21 14:28:03.772367: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-21 14:28:05.118246: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 47.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [4]:
import numpy as np
import pandas as pd 

# ## For tokenization and data pre-processing 
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.pipeline import Pipeline
# ## fro cleaning the data 
import string
import spacy
# from spacy.lang.en.stop_words import STOP_WORDS
# from spacy.lang.en import English


In [5]:
np.random.seed(42)

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
data = pd.read_csv("/content/drive/MyDrive/DSCP Deep Learning/NLP Projects/Session_1/SMSSpamCollection.txt", sep = "\t", names=["label", "message"])
data.head()

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [8]:
data['label'] = data['label'].map({'ham':0,'spam':1})

In [9]:
data['label'].value_counts()

0    4825
1     747
Name: label, dtype: int64

In [10]:
data['label'].value_counts(normalize= True)

0    0.865937
1    0.134063
Name: label, dtype: float64

In [11]:
data_1 = data[data['label']==1]
data_0 = data[data['label']==0]

In [12]:
sample_count = len(data[data['label']==1])
sample_count

747

In [13]:
data_0 = data_0.sample(sample_count, replace=True, random_state=1)
data_0

,label,message
278,0,"Awesome, I'll see you in a bit"
4603,0,"Hey j! r u feeling any better, hopeSo hunny. i..."
1066,0,No my mum went 2 dentist.
3195,0,And you! Will expect you whenever you text! Ho...
3341,0,Like I made him throw up when we were smoking ...
...,...,...
1020,0,Don know..wait i will check it.
5109,0,Oh you got many responsibilities.
703,0,What is important is that you prevent dehydrat...
3842,0,Howz pain.it will come down today.do as i said...


In [14]:
data_balanced = pd.concat([data_0,data_1], axis=0)
data_balanced

,label,message
278,0,"Awesome, I'll see you in a bit"
4603,0,"Hey j! r u feeling any better, hopeSo hunny. i..."
1066,0,No my mum went 2 dentist.
3195,0,And you! Will expect you whenever you text! Ho...
3341,0,Like I made him throw up when we were smoking ...
...,...,...
5537,1,Want explicit SEX in 30 secs? Ring 02073162414...
5540,1,ASKED 3MOBILE IF 0870 CHATLINES INCLU IN FREE ...
5547,1,Had your contract mobile 11 Mnths? Latest Moto...
5566,1,REMINDER FROM O2: To get 2.50 pounds free call...


In [15]:
data = data_balanced

In [16]:
data['label'].value_counts(normalize= True)

0    0.5
1    0.5
Name: label, dtype: float64

Spacy Pre-trained Models  https://spacy.io/models/en

In [17]:
nlp = spacy.load("en_core_web_sm")

In [18]:
stop_words = nlp.Defaults.stop_words
print(stop_words)
punctuations = string.punctuation
punctuations

{'may', 'whereby', 'made', 'neither', 'part', 'there', 'never', 'herself', 'many', 'hereafter', 'else', 'from', '‘ll', '‘re', 'also', "'ll", 'eleven', 'thence', 'per', "'re", 'hundred', '‘ve', 'ourselves', 'six', 'were', 'most', '’m', 'towards', 'nor', 'afterwards', 'twelve', 'even', 'all', 'whose', 'none', 'alone', 'me', 'both', 'always', 'hers', 'but', 'some', 'just', 'over', 'must', 'a', 'next', 'move', 'under', 'once', 'meanwhile', 'otherwise', "'s", '‘d', 'fifteen', 'two', 'be', 'should', 'whereas', 'yourselves', 'throughout', 'wherever', 'amongst', 'enough', 'become', 'ten', 'whoever', 'often', 'it', 'behind', 'though', 'against', 'really', 'already', 'might', 'least', 'on', 'hereupon', 'former', 'with', 'say', 'rather', 'anywhere', 'what', 'further', 'through', 'nobody', 'via', 'him', 'whole', 'herein', 'only', 'other', 'cannot', 'these', 'whereupon', 'fifty', 'take', 'was', 'nothing', 'mostly', 'our', 'someone', 'unless', 'down', 'keep', 'or', 'been', 'and', 'they', 'quite', 'h

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [19]:
# Creating our tokenizer function
def spacy_tokenizer(sentence):
    # Creating our token object, which is used to create documents with linguistic annotations.
    doc = nlp(sentence)

    # print(doc)

    # Lemmatizing each token and converting each token into lowercase
    mytokens = [ word.lemma_.lower().strip() for word in doc ]

    # print(mytokens)

    # Removing stop words
    mytokens = [ word for word in mytokens if word not in stop_words and word not in punctuations ]

    # return preprocessed list of tokens
    return mytokens

In [20]:
sentence = "I am eating apple ?"
spacy_tokenizer(sentence)


['eat', 'apple']

In [21]:

doc = nlp(sentence)
doc

I am eating apple ?

In [22]:
type(sentence),type(doc)

(str, spacy.tokens.doc.Doc)

In [23]:
len(sentence),len(doc)

(19, 5)

In [24]:
sentence

'I am eating apple ?'

In [25]:
for c in sentence:
  print(c)

I
 
a
m
 
e
a
t
i
n
g
 
a
p
p
l
e
 
?


In [26]:
for c in doc:
  print(c.text,c.lemma_, c.is_stop)

I I True
am be True
eating eat False
apple apple False
? ? False


In [ ]:
# doc = nlp(sentence)
# type(doc),type(sentence)
# for token in doc:
#     print(token.text, token.lemma_ , token.is_stop)

In [ ]:
# 

TF_IDF Eeplanation   https://towardsdatascience.com/tf-idf-simplified-aba19d5f5530

In [ ]:
# sentence1 = "I am eating apple, I am happy"
# sentence2 = "I am eating Mango"
# vocabulary = ["I", "am" ,"eating", "apple","Mango" , "happy"]

# one_hot_encoding = []

# sentence1_one_hot = [1 1 1 1 0 ]
# sentence2_one_hot = [1 1 1 0 1 ]

https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html   
https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html#sklearn.feature_extraction.text.TfidfVectorizer

In [44]:
#bow_vector = CountVectorizer(tokenizer = spacy_tokenizer, ngram_range=(1,1))
tfidf_vector = TfidfVectorizer(tokenizer = spacy_tokenizer)

In [34]:
bow_vector.fit_transform(["I am eating apple","I am playing cricket"]).toarray()


array([[1, 0, 1, 0],
       [0, 1, 0, 1]])

In [35]:
bow_vector.vocabulary_

{'eat': 2, 'apple': 0, 'play': 3, 'cricket': 1}

In [30]:
#bow_vector = CountVectorizer(tokenizer = spacy_tokenizer, ngram_range=(1,2))
# tfidf_vector = TfidfVectorizer(tokenizer = spacy_tokenizer)

In [31]:
bow_vector.fit_transform(["I am eating apple","I am playing cricket"]).toarray()

array([[1, 0, 1, 1, 0, 0],
       [0, 1, 0, 0, 1, 1]])

In [32]:
bow_vector.vocabulary_

{'eat': 2,
 'apple': 0,
 'eat apple': 3,
 'play': 4,
 'cricket': 1,
 'play cricket': 5}

In [ ]:
# bow_vector = CountVectorizer(ngram_range=(1,1))
# bow_vector.fit_transform([sentence]).toarray()

In [36]:
from sklearn.model_selection import train_test_split

X = data['message'] # the features we want to analyze
ylabels = data['label'] # the labels, or answers, we want to test against

X_train, X_test, y_train, y_test = train_test_split(X, ylabels, test_size=0.2)

In [37]:
X_train

864     Did u find out what time the bus is at coz i n...
778     Hi its Kate it was lovely to see you tonight a...
2663    Hello darling how are you today? I would love ...
5520                     No. I dont want to hear anything
1227    Reply with your name and address and YOU WILL ...
                              ...                        
2808    December only! Had your mobile 11mths+? You ar...
4073    Loans for any purpose even if you have Bad Cre...
784     You have an important customer service announc...
5278    URGENT! Your Mobile number has been awarded wi...
2774    How come it takes so little time for a child w...
Name: message, Length: 1195, dtype: object

In [38]:
y_train

864     0
778     0
2663    1
5520    0
1227    1
       ..
2808    1
4073    1
784     1
5278    1
2774    1
Name: label, Length: 1195, dtype: int64

https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html

In [39]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()

# Create pipeline using Bag of Words
pipe = Pipeline([('vectorizer', bow_vector),
                 ('classifier', classifier)])

# model generation
pipe.fit(X_train,y_train)

/usr/local/lib/python3.9/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Pipeline(steps=[('vectorizer',
                 CountVectorizer(tokenizer=<function spacy_tokenizer at 0x7f118fa9b5e0>)),
                ('classifier', LogisticRegression())])

In [40]:
from sklearn import metrics
# Predicting with a test dataset
predicted = pipe.predict(X_test)

In [41]:
print("Logistic Regression Accuracy:",metrics.accuracy_score(y_test, predicted))
print("Logistic Regression Precision:",metrics.precision_score(y_test, predicted))
print("Logistic Regression Recall:",metrics.recall_score(y_test, predicted))

Logistic Regression Accuracy: 0.9297658862876255
Logistic Regression Precision: 0.9925925925925926
Logistic Regression Recall: 0.8701298701298701


In [ ]:
#Logistic Regression Accuracy: 0.9297658862876255
#Logistic Regression Precision: 0.9925925925925926
#Logistic Regression Recall: 0.8701298701298701



In [42]:
pipe.predict(['Free entry pass'])


array([0])

In [43]:
pipe.predict(['I am waiting to get response from you'])

array([0])

In [48]:
pipe.predict(['win lottery'])

array([1])

In [45]:
pipe2 = Pipeline([ ('vectorizer', tfidf_vector),
                 ('classifier', classifier)])

# model generation
pipe2.fit(X_train,y_train)
# Predicting with a test dataset
predicted2 = pipe.predict(X_test)

# Model Accuracy
print("Logistic Regression Accuracy:",metrics.accuracy_score(y_test, predicted2))
print("Logistic Regression Precision:",metrics.precision_score(y_test, predicted2))
print("Logistic Regression Recall:",metrics.recall_score(y_test, predicted2))

/usr/local/lib/python3.9/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Logistic Regression Accuracy: 0.9230769230769231
Logistic Regression Precision: 0.901840490797546
Logistic Regression Recall: 0.9545454545454546


In [ ]:
Logistic Regression Accuracy: 0.9230769230769231
Logistic Regression Precision: 0.901840490797546
Logistic Regression Recall: 0.9545454545454546

In [46]:
pipe2.predict(['Free entry pass'])

array([1])

In [47]:
pipe2.predict(['win lottery'])

array([1])

In [49]:
pipe2.predict(['I am waiting to get response from you'])

array([0])